In [111]:
import pandas as pd
import numpy as np
import pyreadr
from statsmodels.regression.linear_model import OLS
from tabulate import tabulate
import statsmodels.formula.api as smf

In [112]:
file_path = "/Users/yaolangzhong/Nottingham_Replication/original_code/"

In [113]:
# FOMC announcements
fomc = pd.read_csv('data/fomc_tight.csv', na_values='.')
fomc.rename(columns={"Date": "date"}, inplace=True)
fomc['date'] = pd.to_datetime(fomc['date'], format='%m/%d/%Y')
fomc['meeting'] = 1
fomc = fomc[fomc['date'].dt.year >= 1994]
fomc = fomc[['date', 'Unscheduled', 'meeting']]

In [114]:
# constant maturity uncertainty measure
mpu = pyreadr.read_r(file_path + "data/mpu.RData")["mpu"]
mpu['date'] = pd.to_datetime(mpu['date'])
data = mpu.copy()
# daily changes in one-year measure
data['dmpu'] = data['mpu10'].diff()
# left join fomc
data = data.merge(fomc, how='left', on='date')
# replace NaN values
data['meeting'] = data['meeting'].fillna(0)
data['Unscheduled'] = data['Unscheduled'].fillna(0)
# create 'ym' column
data['ym'] = data['date'].dt.year*100 + data['date'].dt.month
# select certain columns
data = data[['date', 'meeting', 'Unscheduled', 'dmpu', 'ym']]
# Create 'fomc' DataFrame by filtering 'data'
fomc = data[data['meeting'] == 1]

# Get range of 'date' column
date_range = [fomc['date'].min(), fomc['date'].max()]
print(date_range)
print("# meetings with largest declines in uncertainty (as in Table 3)")
print(data[data['meeting']==1][['date', 'dmpu', 'Unscheduled']].sort_values('dmpu').head(10).to_string(index=False))
print("# meetings with largest increases in uncertainty (as in Table 3)")
print(data[data['meeting']==1][['date', 'dmpu', 'Unscheduled']].sort_values('dmpu', ascending=False).head(5).to_string(index=False))

[Timestamp('1994-02-04 00:00:00'), Timestamp('2020-09-16 00:00:00')]
# meetings with largest declines in uncertainty (as in Table 3)
      date      dmpu  Unscheduled
2011-08-09 -0.137036          0.0
2008-12-16 -0.133744          0.0
1995-07-06 -0.128690          0.0
1998-11-17 -0.123289          0.0
1994-05-17 -0.100870          0.0
2008-11-25 -0.096065          1.0
1998-10-15 -0.082869          1.0
2009-03-18 -0.080559          0.0
2004-06-30 -0.074415          0.0
2008-10-29 -0.074120          0.0
# meetings with largest increases in uncertainty (as in Table 3)
      date     dmpu  Unscheduled
2008-10-08 0.116081          1.0
2004-01-28 0.051986          0.0
1994-04-18 0.038363          1.0
1994-02-04 0.032011          0.0
1995-03-28 0.028046          0.0


In [115]:
## Table C.1 -- changes in MPU/RNV for each contract
results = pyreadr.read_r(file_path + "data/mpu_contracts.RData")["results"]
# Transform the data
data = results[['date', 'exp', 'contract', 'mpu']].copy()
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y')
data['rnv'] = data['mpu'] ** 2
data['j'] = data['contract'].str[2:4].astype(int)
data['ym'] = pd.to_datetime(data['date']).dt.year * 100 + pd.to_datetime(data['date']).dt.month
data['original_index'] = np.arange(len(data))
# Applying group-wise transformations
data = data.groupby('exp').apply(lambda group: group.assign(
    dmpu = group['mpu'].diff(),
    dv = group['rnv'].diff()
))
# Restoring the original order of rows
data = data.sort_values(by='original_index').reset_index(drop=True)
# Dropping the 'original_index' column as it is no longer needed
data = data.drop(columns='original_index')
# Filtering
data = data[((data['ym'] <= 200706) | (data['ym'] >= 200907)) & (data['ym'].between(199401, 202009))]
# Joining with 'fomc'
data = data.merge(fomc[['date', 'meeting', 'Unscheduled']], how='left', on='date')
# Replacing NAs and filtering again
data = data.assign(
    meeting = data['meeting'].fillna(0),
    Unscheduled = data['Unscheduled'].fillna(0)
)
data = data[data['Unscheduled'] == 0].drop(columns='Unscheduled')
# Selecting columns
data = data[['date', 'exp', 'contract', 'mpu', 'rnv', 'j', 'ym', 'dmpu', 'dv', 'meeting']]

In [116]:
J = 6 # number of contracts to include in analysis
print(f"Sample period: {data['date'].min()} - {data['date'].max()}")
print("# Summary statistics for changes in variances on FOMC meetings")

nms = ["Mean", "t-statistic", "Median", "Standard deviation", "Fraction negative", "Observations"]
tbl = pd.DataFrame(index=nms, columns=[f"ED{i+1}" for i in range(J)])

for j in range(1, J+1):
    x = data.loc[(data['j'] == j) & (data['meeting'] == 1), 'dv']
    nobsna = x.isna().sum()
    x.dropna(inplace=True)
    nobs = len(x)
    
    model = OLS(x, np.ones(len(x))).fit(cov_type='HC0')
    tstat = model.params[0]/model.bse[0]

    tbl.loc[:, f"ED{j}"] = [x.mean(), tstat, x.median(), x.std(), (x > 0).mean(), nobs]
tbl = tbl.astype(float).round(3)
print(tbl)
print("# Square roots of (neg) mean/median of variance changes:")
print((-tbl.loc[['Mean', 'Median'], :]).apply(np.sqrt).round(3))
print("# Summary statistics for changes in MPU around FOMC meetings")
tbl2 = pd.DataFrame(index=nms, columns=[f"ED{i+1}" for i in range(J)])
for j in range(1, J+1):
    x = data.loc[(data['j'] == j) & (data['meeting'] == 1), 'dmpu']
    x.dropna(inplace=True)
    nobs = len(x)

    model = OLS(x, np.ones(len(x))).fit(cov_type='HC0')
    tstat = model.params[0]/model.bse[0]
    tbl2.loc[:, f"ED{j}"] = [x.mean(), tstat, x.median(), x.std(), (x > 0).mean(), nobs]

tbl2 = tbl2.astype(float).round(3)
print(tbl2)

Sample period: 1994-01-03 00:00:00 - 2020-09-30 00:00:00
# Summary statistics for changes in variances on FOMC meetings
                        ED1      ED2      ED3      ED4      ED5      ED6
Mean                 -0.009   -0.017   -0.026   -0.033   -0.037   -0.041
t-statistic          -8.861   -9.602   -8.745   -8.286   -7.743   -8.054
Median               -0.004   -0.008   -0.015   -0.016   -0.020   -0.024
Standard deviation    0.013    0.025    0.042    0.056    0.068    0.072
Fraction negative     0.090    0.122    0.188    0.147    0.188    0.186
Observations        177.000  197.000  197.000  197.000  197.000  194.000
# Square roots of (neg) mean/median of variance changes:
          ED1    ED2    ED3    ED4    ED5    ED6
Mean    0.095  0.130  0.161  0.182  0.192  0.202
Median  0.063  0.089  0.122  0.126  0.141  0.155
# Summary statistics for changes in MPU around FOMC meetings
                        ED1      ED2      ED3      ED4      ED5      ED6
Mean                 -0.019   -

In [117]:
# the sink(...) in R is similar to with open(...) as f in Python:
with open("output/table_C1.tex", "w") as f:
    print("& ED1 & ED2 & ED3 & ED4 & ED5 & ED6 \\\\ \n", file=f)
    print("\\midrule \n", file=f)
    print("\\multicolumn{4}{l}{\\emph{Changes in conditional variance}}\\\\ \n", file=f)
    f.write(tabulate(tbl.iloc[0:4, :], tablefmt='latex_raw', showindex=False))
    print("\\midrule \n", file=f)
    print("\\multicolumn{4}{l}{\\emph{Changes in $SRU$}}\\\\ \n", file=f)
    f.write(tabulate(tbl2.iloc[0:4, :], tablefmt='latex_raw', showindex=False))
    print("\\midrule \n", file=f)
    print("Observations" + " ".join(f"& {i}" for i in tbl.loc['Observations', :]) + " \\\\ \n", file=f)
    print("Fraction negative" + " ".join(f"& {i:.2f}" for i in tbl.loc['Fraction negative', :]) + " \\\\ \n", file=f)

In [118]:
## Table C.2: dummy regressions -- compare to days without FOMC
tbl = pd.DataFrame(np.nan, index=range(6), columns=[""]+[f"ED{i+1}" for i in range(J)])
tbl.iloc[:, 0] = ["Constant", "", "FOMC dummy", "", "R^2", "Observations"]
tbl2 = tbl.copy()

In [119]:
## Table C.2: dummy regressions -- compare to days without FOMC
tbl = pd.DataFrame(np.nan, index=range(6), columns=[""]+[f"ED{i+1}" for i in range(J)])
tbl.iloc[:, 0] = ["Constant", "", "FOMC dummy", "", "R^2", "Observations"]
tbl2 = tbl.copy()

def regstats(model):
    b = model.params
    tstat = b / model.HC0_se
    return [f"{b[0]:.3f}", f"[{tstat[0]:.2f}]", f"{b[1]:.3f}", f"[{tstat[1]:.2f}]", f"{model.rsquared:.3f}", len(model.resid)]

for j in range(J):
    regdat = data.loc[(data['j'] == j + 1)]
    mod = smf.ols('dv ~ meeting', data=regdat).fit()
    tbl.iloc[:, j+1] = regstats(mod) 
    mod2 = smf.ols('dmpu ~ meeting', data=regdat).fit()
    tbl2.iloc[:, j+1] = regstats(mod2)


In [120]:
print("# Change in variance due to FOMC meetings (if no jumps on other days, negative is mean jump variance)")
print(tbl)
print("# Jump volatility - square root of negative dummy coefficient:")
print(round((-tbl.iloc[2,1:].astype(float))**0.5, 3))
print("# Decline in MPU due to FOMC meetings")
print(tbl2)

with open("output/table_C2.tex", "w") as f:
    f.write("& ED1 & ED2 & ED3 & ED4 & ED5 & ED6 \\\\ \n")
    f.write("\\midrule \n")
    f.write("\\multicolumn{4}{l}{\\emph{Changes in conditional variance}}\\\\ \n")
    # Assuming that you have converted the table 'tbl' to LaTeX using to_latex() and stored it in tbl_latex
    tbl_latex = tbl.iloc[0:5,:].to_latex(index=False, header=False)
    f.write(tbl_latex)
    jump_vols = [f"& {vol:.3f}" for vol in ((-tbl.iloc[2,1:].astype(float))**0.5)]
    f.write("\\emph{Memo}: estimated jump vol." + ''.join(jump_vols) + " \\\\ \n")
    f.write("\\midrule \n")
    f.write("\\multicolumn{4}{l}{\\emph{Changes in $SRU$}}\\\\ \n")
    # Assuming that you have converted the table 'tbl2' to LaTeX using to_latex() and stored it in tbl2_latex
    tbl2_latex = tbl2.iloc[0:5,:].to_latex(index=False, header=False)
    f.write(tbl2_latex)
    f.write("\\midrule \n")
    observations = ["& {}".format(obs) for obs in tbl.iloc[5,1:]]
    f.write("Observations" + ''.join(observations) + " \\\\ \n")

# Change in variance due to FOMC meetings (if no jumps on other days, negative is mean jump variance)
                      ED1       ED2      ED3      ED4      ED5      ED6
0      Constant    -0.001    -0.002   -0.003   -0.004   -0.005   -0.005
1                [-11.85]  [-10.97]  [-9.54]  [-8.64]  [-7.50]  [-6.15]
2    FOMC dummy    -0.007    -0.015   -0.023   -0.028   -0.032   -0.037
3                 [-7.44]   [-8.22]  [-7.55]  [-7.12]  [-6.70]  [-7.09]
4           R^2     0.028     0.022    0.019    0.015    0.013    0.012
5  Observations      4922      6228     6230     6230     6230     6176
# Jump volatility - square root of negative dummy coefficient:
ED1    0.084
ED2    0.122
ED3    0.152
ED4    0.167
ED5    0.179
ED6    0.192
Name: 2, dtype: float64
# Decline in MPU due to FOMC meetings
                      ED1       ED2       ED3       ED4      ED5      ED6
0      Constant    -0.003    -0.003    -0.003    -0.003   -0.002   -0.002
1                [-15.88]  [-13.07]  [-11.3

In [137]:
# Rename 'meeting' to 'FOMC'
regdat = data.rename(columns={'meeting': 'FOMC'})
regdat['original_index'] = np.arange(len(regdat))
# Create lagged variables D1 to D25 and sum variables W1 to W5 within each 'exp' group
regdat = regdat.sort_values(by=['exp', 'date'])
regdat = regdat.assign(**{f'D{i}': regdat.groupby('exp')['FOMC'].shift(i) for i in range(1, 26)})
for i in range(1, 6):
    start, end = 5*(i-1)+1, 5*i+1
    regdat[f'W{i}'] = regdat[[f'D{j}' for j in range(start, end)]].sum(axis=1)
# Restoring the original order of rows
regdat = regdat.sort_values(by='original_index').reset_index(drop=True)
# Dropping the 'original_index' column as it is no longer needed
regdat = regdat.drop(columns='original_index')


In [160]:
# Define variables and formula
regs = ["FOMC"] + [f"W{i}" for i in range(1, 6)]
fmla = 'dmpu ~ ' + ' + '.join(regs)
# Initialize DataFrame for storing regression results
index_values = [val for sublist in [[reg, ''] for reg in ["Constant"] + regs] for val in sublist] + ["R^2", "Observations"]
tbl = pd.DataFrame(index=index_values, columns=[f"ED{i}" for i in range(1, J+1)])
def regstats(mod):
    b = mod.params
    tstat = b / np.sqrt(mod.HC0_se)
    return [f"{b_val:.2f}" if idx != 'R^2' else f"{b_val:.3f}" for idx, b_val in b.items()] + [f"[{t_val:.2f}]" for t_val in tstat] + [f"{mod.rsquared:.3f}", len(mod.resid)]


In [162]:
for j in range(1, J+1):
    mod = smf.ols(formula=fmla, data=regdat[regdat['j'] == j]).fit()
    tbl[f"ED{j}"] = regstats(mod)

In [169]:
j = 1
temp=regdat[regdat['j'] == j]
temp.head(10)

,date,exp,contract,mpu,rnv,j,ym,dmpu,dv,FOMC,...,D21,D22,D23,D24,D25,W1,W2,W3,W4,W5
0,1994-01-03,1994-03-14,ED1,0.276462,0.076431,1,199401,-0.006554,-0.003667,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
6,1994-01-04,1994-03-14,ED1,0.264381,0.069897,1,199401,-0.012081,-0.006534,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
12,1994-01-05,1994-03-14,ED1,0.264360,0.069886,1,199401,-0.000021,-0.000011,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
18,1994-01-06,1994-03-14,ED1,0.266483,0.071013,1,199401,0.002123,0.001127,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
24,1994-01-07,1994-03-14,ED1,0.238620,0.056940,1,199401,-0.027863,-0.014074,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
30,1994-01-10,1994-03-14,ED1,0.202347,0.040944,1,199401,-0.036273,-0.015995,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
36,1994-01-11,1994-03-14,ED1,0.202339,0.040941,1,199401,-0.000007,-0.000003,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
42,1994-01-12,1994-03-14,ED1,0.204719,0.041910,1,199401,0.002379,0.000969,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
48,1994-01-13,1994-03-14,ED1,0.204296,0.041737,1,199401,-0.000422,-0.000173,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
54,1994-01-14,1994-03-14,ED1,0.202310,0.040929,1,199401,-0.001986,-0.000808,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


In [163]:
tbl

,ED1,ED2,ED3,ED4,ED5,ED6
Constant,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00
,-0.02,-0.01,-0.02,-0.01,-0.01,-0.01
FOMC,0.00,0.00,0.00,0.00,0.00,0.00
,0.00,0.00,0.00,0.00,0.00,0.00
W1,-0.00,-0.00,-0.00,0.00,-0.00,-0.00
,-0.00,0.00,0.00,0.00,0.00,-0.00
W2,-0.00,0.00,0.00,0.00,0.00,0.00
,[-0.14],[-0.15],[-0.15],[-0.15],[-0.13],[-0.11]
W3,[-0.39],[-0.38],[-0.36],[-0.34],[-0.32],[-0.32]
,[0.01],[0.07],[0.08],[0.08],[0.07],[0.05]


In [173]:
with open('output/table_D5.tex', 'w') as f:
    f.write("& ED1 & ED2 & ED3 & ED4 & ED5 & ED6 \\\\ \n")
    f.write("\\midrule \n")
    f.write(tbl.iloc[:-2, :].to_latex(header=False, index=False))
    f.write("\\midrule \n")
    f.write(tbl.iloc[-2:, :].to_latex(header=False, index=False))